In [128]:
!pip install pandas

import pandas as pd
import numpy as np
import requests # library to handle requests
import json # library to handle JSON files
from pandas.io.json import json_normalize
from datetime import datetime
import os.path 
from os import path

In [173]:
#returns weather json data from weatherstack api
def request_weather_json(key, date, location):
    params = {
        'access_key': key,
        'query': location,
        'historical_date': date,
        'hourly': '1',
        'interval': '1'
        }

    api_result = requests.get('https://api.weatherstack.com/historical', params)
    api_response = api_result.json()
    #print(api_response['location'])
    return api_response

In [135]:
#Formats weather json data and outputs to detailed_weather_report.csv
def append_weather_data(json):
    json_historical = json_normalize(data=json['historical'][date], errors='ignore')
    detailed_weather_report = json_normalize(data=json_historical['hourly'][0], meta=['day'], meta_prefix='pre', errors='ignore')
    detailed_weather_report['location'] = json['location']['name']
    detailed_weather_report['date'] = date
    
    if path.exists("detailed_weather_report.csv"):
        detailed_weather_report.to_csv('detailed_weather_report.csv', mode='a', header=False)
    else:
        detailed_weather_report.to_csv('detailed_weather_report.csv')

In [176]:
def create_report():
    detailed_weather_df = pd.read_csv("detailed_weather_report.csv")
    weather_df = detailed_weather_df[detailed_weather_df['Unnamed: 0'].between(9,17)]
    weather_df['precip'].replace({0:np.nan}, inplace=True)
    weather_df = weather_df.groupby(['location','date']).agg({'precip':'count', 'temperature': 'max'}).reset_index().rename(columns={'precip':'Hours Rain', 'temperature':'Max Temp'})
    weather_df.to_csv('weather_report.csv')
    weather_df

In [177]:
key = '8c5dad538cf97d8347dbfda0e9369f25'
dates = ['2020-01-02', '2020-01-03']
locations = ['London, United Kingdom', 'Edinburgh, United Kingdom']

for date in dates:
    for location in locations:
        json = request_weather_json(key, date, location)
        append_weather_data(json)
        
create_report()